In [6]:
import json
import os
import base64
from PIL import Image
from io import BytesIO

def save_images_from_notebook(notebook_data, output_folder="images"):
    """
    Extracts image outputs from the notebook and saves them to a folder.
    Returns a list of image file paths.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_paths = []
    for cell in notebook_data.get("cells", []):
        for output in cell.get("outputs", []):
            if 'data' in output and 'image/png' in output['data']:
                image_data = output['data']['image/png']
                image_path = os.path.join(output_folder, f"image_{len(image_paths)}.png")
                with open(image_path, "wb") as img_file:
                    img_file.write(base64.b64decode(image_data))
                image_paths.append(image_path)
    
    return image_paths

def convert_images_to_markdown(image_paths):
    """
    Converts image paths to Markdown image syntax.
    """
    markdown_images = ""
    for image_path in image_paths:
        # Assuming images are saved in an 'images' folder
        markdown_images += f"![Image]({image_path})\n\n"
    return markdown_images

def copy_notebook_to_readme(notebook_name, readme_name, output_folder="images"):
    # Ensure that the notebook and README file exist in the current directory
    if not os.path.isfile(notebook_name):
        print(f"Error: The notebook {notebook_name} does not exist.")
        return
    
    if not os.path.isfile(readme_name):
        print(f"Error: The README file {readme_name} does not exist.")
        return
    
    # Read the notebook content
    with open(notebook_name, 'r', encoding='utf-8') as notebook_file:
        notebook_data = json.load(notebook_file)
    
    # Extract code and markdown cells
    cells = notebook_data.get("cells", [])
    content = ""
    
    for cell in cells:
        if cell["cell_type"] == "code":
            content += "```python\n" + "".join(cell["source"]) + "\n```\n"
        elif cell["cell_type"] == "markdown":
            content += "".join(cell["source"]) + "\n\n"
    
    # Save images and include in the content
    image_paths = save_images_from_notebook(notebook_data, output_folder)
    markdown_images = convert_images_to_markdown(image_paths)
    
    content += markdown_images

    # Append to README file
    with open(readme_name, 'a', encoding='utf-8') as readme_file:
        readme_file.write(content)
        print(f"Notebook content, including images, has been copied to {readme_name}.")

# Usage
notebook_name = "idex.ipynb"  # Change this to your notebook's name
readme_name = "README.md"  # This is the README file in the same folder
copy_notebook_to_readme(notebook_name, readme_name)

Notebook content, including images, has been copied to README.md.
